In [1]:
import numpy as np
from scipy.stats import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.datasets import make_classification

In [ ]:
def custom_log_loss(y_true, y_pred):
    """
    Вычисление логарифмической функции потерь (log-loss) вручную.
    
    :param y_true: Список или массив истинных классов (0 или 1 для бинарной классификации).
    :param y_pred: Список или массив предсказанных вероятностей для класса 1.
    :return: Значение log-loss.
    """
    # Преобразуем входные данные в массивы NumPy для удобства вычислений
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Добавляем защиту от 0 и 1, чтобы избежать проблем с log(0)
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    
    # Формула log-loss для бинарной классификации
    loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    
    return loss


def mcfadden_pseudo_r2(y_true, y_pred_prob):
    """
    Вычисление McFadden Pseudo R².
    
    :param y_true: Массив истинных меток (0 или 1).
    :param y_pred_prob: Массив предсказанных вероятностей для класса 1.
    :return: Значение McFadden Pseudo R².
    """
    # Логарифм правдоподобия модели
    ll_model = - custom_log_loss(y_true, y_pred_prob)
    
    # Логарифм правдоподобия базовой модели (всегда предсказывает вероятность среднего значения)
    mean_prob = np.mean(y_true)
    ll_null = - custom_log_loss(y_true, np.full_like(y_true, mean_prob))
    
    # McFadden Pseudo R²
    r2_mcfadden = 1 - (ll_model / ll_null)
    return r2_mcfadden


def calculate_p_value_mcfadden(y_true, y_pred_prob, num_predictors):
    """
    Рассчёт p-value для критерия McFadden с использованием LR-теста.
    
    :param y_true: Массив истинных меток (0 или 1).
    :param y_pred_prob: Массив предсказанных вероятностей для класса 1.
    :param num_predictors: Число предикторов в модели (k).
    :return: p-value и статистика chi2.
    """
    # Логарифм правдоподобия модели
    ll_model = -custom_log_loss(y_true, y_pred_prob)
    
    # Логарифм правдоподобия нулевой модели
    mean_prob = np.mean(y_true)
    ll_null = -custom_log_loss(y_true, np.full_like(y_true, mean_prob))
    
    # LR-статистика
    chi2_stat = 2 * (ll_model - ll_null)
    
    # Число степеней свободы
    df = num_predictors
    
    # p-value
    p_value = chi2.sf(chi2_stat, df)
    
    return p_value, chi2_stat

In [ ]:
n_features = 7

X, y = make_classification(n_samples=1000, n_features=n_features, random_state=42)

model = LogisticRegression()
model.fit(X, y)
y_pred_prob = model.predict_proba(X)[:, 1]

r2_mcfadden = mcfadden_pseudo_r2(y, y_pred_prob)
print(f"McFadden Pseudo R²: {r2_mcfadden:.4f}")

McFadden Pseudo R²: 0.9802
